In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=37d7c83d2bd0f46b9f7e5371e95e5f4c191082ce331b6781a4ee0ae6fc684df0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [624 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,203 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,430 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,010 kB]
Get:13 http://secu

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_view')

In [6]:
spark.sql("SELECT * FROM home_view LIMIT 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query1 = """
(SELECT ROUND(AVG(price), 2), YEAR(date) FROM home_view
WHERE bedrooms==4
GROUP BY YEAR(date))
"""
spark.sql(query1).show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
+--------------------+----------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query2 = """
(SELECT ROUND(AVG(price), 2), date_built FROM home_view
WHERE bedrooms == 3 and bathrooms == 3
GROUP BY date_built)
"""
spark.sql(query2).show()



+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|            288770.3|      2015|
|           295962.27|      2013|
|           290852.27|      2014|
|           293683.19|      2012|
|           290555.07|      2016|
|           292859.62|      2010|
|           291117.47|      2011|
|           292676.79|      2017|
+--------------------+----------+



In [11]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query3 = """
(SELECT ROUND(AVG(price), 2), date_built FROM home_view
WHERE bedrooms == 3 and bathrooms == 3 and floors == 2 and sqft_living >= 2000
GROUP BY date_built)
"""
spark.sql(query3).show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           297609.97|      2015|
|           303676.79|      2013|
|           298264.72|      2014|
|           307539.97|      2012|
|            293965.1|      2016|
|           285010.22|      2010|
|           276553.81|      2011|
|           280317.58|      2017|
+--------------------+----------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

#WHERE keyword cannot be used with aggregate functions, so we are using HAVING instead
query4 = """
(SELECT ROUND(AVG(price), 2), view FROM home_view
GROUP BY view
HAVING ROUND(AVG(price),2) >= 350000)
"""
spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|          1053472.79|  81|
|          1058802.78|  76|
|          1137372.73|  91|
|          1026006.06|  93|
|           733780.26|  52|
|          1070444.25|  86|
|           1033536.2|  94|
|            734340.5|  57|
|           798684.82|  54|
|          1017815.92|  96|
|           970402.55|  92|
|           767036.67|  64|
|           746877.59|  61|
|          1031719.35|  88|
|           780914.67|  72|
|            791453.0|  59|
+--------------------+----+
only showing top 20 rows

--- 0.7870583534240723 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_view")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_view')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query5 = """
(SELECT ROUND(AVG(price), 2), view FROM home_view
GROUP BY view
HAVING ROUND(AVG(price),2) >= 350000)
"""
spark.sql(query5).show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|          1053472.79|  81|
|          1058802.78|  76|
|          1137372.73|  91|
|          1026006.06|  93|
|           733780.26|  52|
|          1070444.25|  86|
|           1033536.2|  94|
|            734340.5|  57|
|           798684.82|  54|
|          1017815.92|  96|
|           970402.55|  92|
|           767036.67|  64|
|           746877.59|  61|
|          1031719.35|  88|
|           780914.67|  72|
|            791453.0|  59|
+--------------------+----+
only showing top 20 rows

--- 0.7199506759643555 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")

In [17]:
# 11. Read the parquet formatted data.
home_df_par=spark.read.parquet('home_partitioned')

In [18]:
# 12. Create a temporary table for the parquet data.
home_df_par.createOrReplaceTempView('home_temp_parquet')

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query6 = """
(SELECT ROUND(AVG(price), 2), view FROM home_temp_parquet
GROUP BY view
HAVING ROUND(AVG(price),2) >= 350000)
"""
spark.sql(query6).show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|          1053472.79|  81|
|          1058802.78|  76|
|          1137372.73|  91|
|          1026006.06|  93|
|           733780.26|  52|
|          1070444.25|  86|
|           1033536.2|  94|
|            734340.5|  57|
|           798684.82|  54|
|          1017815.92|  96|
|           970402.55|  92|
|           767036.67|  64|
|           746877.59|  61|
|          1031719.35|  88|
|           780914.67|  72|
|           771153.32|  55|
+--------------------+----+
only showing top 20 rows

--- 0.6970400810241699 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_view")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_view')


False